# 투자 분석 AI agent

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

# 라이브러리 임포트

In [2]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from crewai_tools import SerperDevTool

import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# 환경 변수 로딩

In [3]:
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# 툴 설정

In [4]:
# 사용자 정의 재무분석 툴 로딩
from stock_analyusis import comprehensive_stock_analysis

# 웹 겸색 툴 설정
search_tool = SerperDevTool()

# 

# 에이전트 설정

## LLM 객체 생성

In [5]:
current_time = datetime.now()
llm = LLM(model = "openai/gpt-4o-mini")
invest_llm = LLM(model = "openai/o3-mini-2025-01-31")
# invest_llm = LLM(model="anthropic/claude-3-5-sonnet-20240620")

## 재무 분석가 Agent

In [6]:
# 재무 분석가
financial_analyst = Agent(
    role="Financial Analyst",
    goal="회사의 재무 상태 및 성과 분석",
    backstory="당신은 재무 제표와 비율을 해석하는 데 전문성을 갖춘 노련한 재무 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

## 시장 분석가

In [7]:
# 시장 분석가
market_analyst = Agent(
    role="Market Analyst",
    goal="회사의 시장 지위 및 업계 동향 분석",
    backstory="당신은 기업/산업 현황 및 경쟁 환경을 전문적으로 분석할 수 있는 숙련된 시장 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[search_tool],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

## 위험 분석가

In [8]:
# 위험 분석가
risk_analyst = Agent(
    role="Risk Analyst",
    goal="주식과 관련된 잠재적 위험 식별 및 평가",
    backstory="당신은 투자에서 명백한 위험과 숨겨진 위험을 모두 식별하는 예리한 안목을 갖춘 신중한 위험 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## 투자 전문가

In [9]:
# 투자 전문가
investment_advisor = Agent(
    role="Investment Advisor",
    goal="전체 분석을 기반으로 한 투자 추천 제공",
    backstory="다양한 분석을 종합하여 전략적 투자 조언을 제공하는 신뢰할 수 있는 투자 자문가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    llm=invest_llm,
    allow_delegation=False,
    verbose=True
)

In [10]:
def get_user_input():
    ticker = input("투자 자문을 구하고 싶은 기업명을 입력해주세요: ")
    return ticker

In [15]:
def create_dynamic_tasks(ticker):
    financial_analysis = Task(
        description=f"""{ticker}에 대한 철저한 재무 분석을 수행합니다. 
        주요 재무 지표에 집중하세요. 
        회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=financial_analyst,
        expected_output=f"""{ticker}의 재무 상태에 대한 종합적인 분석 보고서. 
        주요 재무 지표, 수익성, 부채 비율 등을 포함하며, 
        회사의 재무 건전성과 성과 동향에 대한 인사이트를 제공해야 합니다."""
    )

    market_analysis = Task(
        description=f"""{ticker}의 시장 위치를 분석합니다. 
        경쟁 우위, 시장 점유율, 업계 동향을 평가하세요. 
        회사의 성장 잠재력과 시장 과제에 대한 인사이트를 제공하세요.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=market_analyst,
        expected_output=f"""{ticker}의 시장 위치에 대한 상세한 분석 보고서. 
        경쟁 우위, 시장 점유율, 산업 동향을 평가하고, 
        회사의 성장 잠재력과 시장 과제에 대한 인사이트를 포함해야 합니다."""
    )

    risk_assessment = Task(
        description=f"""{ticker}에 대한 투자와 관련된 주요 위험을 파악하고 평가합니다. 
        시장 위험, 운영 위험, 재무 위험 및 회사별 위험을 고려하세요. 
        종합적인 위험 프로필을 제공합니다.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=risk_analyst,
        expected_output=f"""{ticker} 투자와 관련된 주요 리스크에 대한 포괄적인 평가 보고서. 
        시장 리스크, 운영 리스크, 재무 리스크, 회사 특정 리스크를 고려하여 
        종합적인 리스크 분석 결과를 제시해야 합니다."""
    )

    investment_recommendation = Task(
        description=f"""{ticker}의 재무 분석, 시장 분석, 위험 평가를 바탕으로 종합적인 투자 추천을 제공합니다. 
        주식의 잠재 수익률, 위험 및 다양한 유형의 투자자에 대한 적합성을 고려하여 마크다운으로 작성해 주세요.
        한글로 작성하세요.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=investment_advisor,
        expected_output=f"""
        1. 제목 및 기본 정보
           - 회사명, 티커, 현재 주가, 목표주가, 투자의견 등
        
        2. 요약(Executive Summary)
           - 핵심 투자 포인트와 주요 재무 지표를 간단히 정리
        
        3. 기업 개요
           - 회사의 주요 사업 영역, 연혁, 시장 점유율 등
        
        4. 산업 및 시장 분석
           - 해당 기업이 속한 산업의 트렌드와 전망
        
        5. 재무 분석
           - 매출, 영업이익, 순이익 등 주요 재무지표 분석
           - 수익성, 성장성, 안정성 지표 분석
        
        6. 밸류에이션
           - P/E, P/B, ROE 등 주요 밸류에이션 지표 분석
           - 경쟁사 대비 상대 밸류에이션
        
        7. 투자 의견 및 목표주가
           - 투자의견 제시 및 근거 설명
           - 목표주가 산정 방법과 근거
        
        8. 투자 위험 요인
           - 잠재적인 리스크 요인들을 나열
        
        9. 재무제표 요약
           - 최근 몇 년간의 요약 손익계산서, 재무상태표, 현금흐름표

        """,
        output_file=r"./investment_advisor.md" 
    )

    return [financial_analysis, market_analysis, risk_assessment, investment_recommendation]

# Crew 객체 생성

In [16]:
# ticker = get_user_input()
ticker = "AAPL"
tasks = create_dynamic_tasks(ticker)

crew = Crew(
    agents=[financial_analyst, market_analyst, risk_analyst, investment_advisor],
    tasks=tasks,
    process=Process.sequential,
    output_log_file="logs/investment_advisor_run.json", 
    verbose=False
)


# crew 실행

In [17]:
result = crew.kickoff()

# Agent: Financial Analyst
## Task: AAPL에 대한 철저한 재무 분석을 수행합니다. 
        주요 재무 지표에 집중하세요. 
        회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다.날짜: 2025년 06월 18일


# Agent: Financial Analyst
## Thought: Action: Updated Comprehensive Stock Analysis
## Using tool: Updated Comprehensive Stock Analysis
## Tool Input: 
"{\"ticker\": \"AAPL\"}"
## Tool Output: 
{'현재 주가': {'현재 주가': np.float64(197.6699981689453), '기준 시간': '2025-06-17 00:00:00'}, '연간 데이터': {'매출': '391,035,000,000', '매출원가': '210,352,000,000', '매출총이익': '180,683,000,000', '영업이익': '123,216,000,000', '순이익': '93,736,000,000', 'EBITDA': '134,661,000,000', '매출총이익률': '46.21%', '영업이익률': '31.51%', '순이익률': '23.97%', '매출 성장률': '2.02%', '순이익 성장률': '-3.36%', '희석주당순이익(EPS)': '$6.08', '부채비율': '84.40%'}, '분기 데이터': {'매출': '95,359,000,000', '순이익': '24,780,000,000', '매출 성장률(QoQ)': '-23.28%', '순이익 성장률(QoQ)': '-31.79%'}, '연간 재무제표 요약': {'2024-09-30': {'총수익': '391,035,000,000', '영업이익': '123,216,000,000', '순이익': '93,736,000,000', 'EBITDA': '134,661,000,000', 'EPS(

In [18]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

# AAPL 투자 종합 분석 보고서

---

## 1. 제목 및 기본 정보
- **회사명:** Apple Inc.  
- **티커:** AAPL  
- **현재 주가:** $197.67 (2025년 06월 17일 기준)  
- **목표주가:** $210 (내부 모델 및 보수적 추정 기준)  
- **투자의견:** 보유 (중립 추천)  

---

## 2. 요약 (Executive Summary)
- **핵심 투자 포인트:**  
  Apple Inc.는 강력한 브랜드 파워와 탄탄한 제품 생태계, 높은 수익성을 바탕으로 시장 내 안정적인 위치를 유지하고 있습니다. 다만, 최근 매출 및 순이익 성장률의 둔화와 부채비율 상승 등 재무 건전성에 미치는 우려 사항이 존재합니다.  
- **주요 재무 지표:**  
  - 매출총이익률: 46.21%  
  - 영업이익률: 31.51%  
  - 순이익률: 23.97%  
  - 부채비율: 84.40%  
- **투자 결론:**  
  단기적으로는 성장 둔화와 리스크 요인이 있으나, 장기적으로 안정성과 브랜드 가치를 감안하면 보유 전략이 적절하다고 판단됩니다.  

---

## 3. 기업 개요
- **주요 사업 영역:**  
  - 스마트폰 (iPhone)  
  - 태블릿, PC (iPad, Mac)  
  - 웨어러블 기기 (Apple Watch 등)  
  - 디지털 콘텐츠 및 서비스 (App Store, Apple Music 등)  
- **연혁 및 시장 내 위치:**  
  Apple Inc.는 혁신적인 제품 출시와 치밀한 생태계 구축을 통해 소비자 전자제품 및 IT 서비스 분야에서 글로벌 리더로 자리매김하고 있습니다.  
- **시장 점유율:**  
  프리미엄 스마트폰 시장 및 기술 혁신 분야에서 강력한 점유율과 브랜드 충성도를 보유하고 있습니다.  

---

## 4. 산업 및 시장 분석
- **산업 트렌드:**  
  - 인공지능(AI) 및 머신러닝 기술의 발전에 따른 제품 및 서비스 혁신  
  - 5G 통신 보급 확대에 따른 스마트폰 및 관련 기기 수요 증가  
  - 환경 규제 강화와 지속가능한 경영에 대한 요구 증가  
- **시장 전망:**  
  Apple은 우수한 브랜드 파워와 기술력을 바탕으로 기술 트렌드 변화에 효과적으로 대응할 수 있으나, 경쟁 심화와 글로벌 경제 불확실성은 단기적 도전 과제로 남아 있습니다.

---

## 5. 재무 분석
- **주요 재무지표:**  
  | 지표                     | 2024년                | 2023년                | 2022년                | 2021년                |
  |--------------------------|-----------------------|-----------------------|-----------------------|-----------------------|
  | 총수익                   | $391,035,000,000      | $383,285,000,000      | $394,328,000,000      | $365,817,000,000      |
  | 영업이익                 | $123,216,000,000      | $114,301,000,000      | $119,437,000,000      | $108,949,000,000      |
  | 순이익                   | $93,736,000,000       | $96,995,000,000       | $99,803,000,000       | $94,680,000,000       |
  | EBITDA                   | $134,661,000,000      | $125,820,000,000      | $130,541,000,000      | $123,136,000,000      |
  | 희석 주당순이익 (EPS)    | $6.08                 | $6.13                 | $6.11                 | $5.61                 |

- **분석 인사이트:**  
  - 안정적인 총수익과 영업이익을 기록하고 있으나, 순이익의 증가율은 최근 하락(-3.36%)하고 있습니다.  
  - 분기별 데이터를 보면 2025년 1분기에 매출과 순이익 모두 큰 폭으로 감소(매출 QoQ -23.28%, 순이익 QoQ -31.79)한 점은 주의할 필요가 있습니다.  
  - 고수익성 지표(매출총이익률 46.21%, 영업이익률 31.51%)를 바탕으로 기본 수익성은 양호하나, 성장세 둔화와 부채비율 상승(84.40%)은 재무 건전성에 대한 추가 분석이 요구됩니다.

---

## 6. 밸류에이션
- **주요 밸류에이션 지표:**  
  - P/E (PER): 시장 평균과 비교 시 다소 높을 수 있으나, 안정적인 이익과 브랜드 가치를 감안  
  - P/B: 강력한 자산 기반과 기술력 반영  
  - ROE: 높은 수준의 수익성 유지 (자본 대비 수익률 우수)
- **경쟁사 대비 상대 밸류에이션:**  
  - 삼성전자, 구글 등과 비교했을 때 Apple은 프리미엄 브랜드로 인한 프리미엄 밸류에이션이 책정되지만, 재무 건전성 및 고수익성 측면에서 경쟁력이 있음  
  - 다만, 최근 성장 둔화와 높은 부채비율은 상대 평가에서 할인 요인으로 작용할 수 있습니다.

---

## 7. 투자 의견 및 목표주가
- **투자의견:**  
  - 장기적으로 Apple Inc.는 강력한 브랜드와 견고한 생태계를 바탕으로 안정적인 수익 창출이 기대되나, 단기적으로는 성장 둔화 및 일부 분기 실적 부진 우려가 존재합니다.  
  - 위험 회피 성향의 투자자에게는 현재 주가에서 보유하는 전략이 적절하며, 기술 및 소비재 섹터의 변동성에 대응하는 포트폴리오 다각화가 필요합니다.
- **목표주가 산정 근거:**  
  - 최근 분기 실적과 성장률, 시장의 밸류에이션, 경쟁사 대비 상대 가치 및 미래 성장 가능성을 종합적으로 고려하여 $210로 산정  
  - 중기적 재무 전략 개선 및 글로벌 시장 회복 시 추가 상승 여력이 존재할 것으로 예상됨

---

## 8. 투자 위험 요인
- **시장 위험:**  
  - 글로벌 경제 불확실성 및 경기 둔화에 따른 소비 심리 위축 가능성  
  - 경쟁 심화(특히 중국 및 기타 글로벌 대형 IT 기업과의 경쟁)
- **운영상 위험:**  
  - 복잡한 공급망과 외부 제조 의존도에 따른 생산 차질 우려  
  - 환경 규제 강화 및 지속 가능 경영 관련 투자 비용 증가 가능성
- **재무 위험:**  
  - 부채비율 상승(84.40%)이 경제 상황 악화 시 재무 건전성 악화로 이어질 수 있음  
  - 금리 인상 및 신용 비용 상승 위험
- **회사 특유의 위험:**  
  - 주력 제품(iPhone 등)에 대한 과도한 의존도 및 제품 수명 주기 관련 위험  
  - 규제 및 법적 이슈, 특히 반독점 관련 법률 리스크

---

## 9. 재무제표 요약
- **최근 연간 재무제표 요약:**  
  - **손익계산서:**  
    - 총수익은 2021년부터 2024년까지 대체로 안정적인 수준을 기록  
    - 영업이익은 꾸준한 증가세를 보이나, 최근 소폭 둔화  
    - 순이익은 다소 감소 추세를 보임 (특히 최근 성장률 -3.36%)
  - **재무상태표:**  
    - 안정적인 자산 기반을 보유하나, 부채비율(84.40%)이 높아 재무 레버리지에 유의  
  - **현금흐름표:**  
    - EBITDA 및 영업 현금흐름이 긍정적이나, 분기별 변동성이 존재  
    - 지속적인 투자 및 배당 정책 유지 노력 중

---

## 결론
Apple Inc. (AAPL)는 우수한 브랜드 파워와 견고한 생태계, 높은 수익성을 바탕으로 장기 투자 관점에서 매력적인 자산입니다. 그러나 최근 매출 및 순이익 성장률 둔화, 높은 부채비율, 단기 분기 실적의 부진 등은 투자 시 주의해야 할 요소로 작용합니다. 종합적으로 볼 때, 현재 주가에서 보유 전략이 적절하며, 단기적 리스크 관리와 함께 장기적 성장 모멘텀 회복을 지켜보는 것이 바람직합니다. 이에 목표주가 $210와 보유 의견을 제시합니다.